# **Lab 10: Prompt Engineering with TheMealDB**
### *Zero-Shot • Few-Shot • Chain-of-Thought • Tool Use*

---

In this lab, you will explore how different prompting strategies affect an LLM’s output.  
You will use **TheMealDB API**  again as the factual data source and apply:

- **Zero-shot prompting**
- **Few-shot prompting**
- **Chain-of-thought reasoning (CoT)**


---

### **This lab reinforces the Golden Rule**

> **LLM = reasoning + language**  
> **API / Data = facts + evidence**


##Preparation: Setting Up TheMealDB Data

Before starting the prompt engineering exercises, we need to prepare a small, clean dataset from **TheMealDB API**.


- Fetch a few meal records using an API call.
- Keep only useful columns (name, category, area, instructions, and ingredients).
- Create a tidy `ingredients_list` column that combines all ingredient and measure fields.
- Convert the final data into JSON format to use as LLM input.

---

### TheMealDB Endpoint

We will use the **search endpoint**, which lets us look up meals by a keyword.

| Task | Endpoint URL | Parameter | Example |
|------|---------------|------------|----------|
| Search meals by name | `https://www.themealdb.com/api/json/v1/1/search.php` | `s` | `"chicken"` |

---

### Why this matters

Preparing clean JSON data ensures the LLM only sees **factual, structured information** from the API.  
This helps us apply the **Golden Rule** of this lab:

> **LLM = reasoning + language**  
> **API / Data = facts + evidence**



In [1]:
import requests
import pandas as pd

# 1. Fetch data from TheMealDB using ?s= search

BASE_URL = "https://www.themealdb.com/api/json/v1/1/search.php"

params = {
    "s": "chicken" # change "chicken" to search anything else
}
response = requests.get(BASE_URL, params)
data = response.json()

# Extract meals list
meals = data.get("meals", [])
df = pd.json_normalize(meals)
df.head()



,idMeal,strMeal,strMealAlternate,strCategory,strArea,strInstructions,strMealThumb,strTags,strYoutube,strIngredient1,...,strMeasure15,strMeasure16,strMeasure17,strMeasure18,strMeasure19,strMeasure20,strSource,strImageSource,strCreativeCommonsConfirmed,dateModified
0,52795,Chicken Handi,None,Chicken,Indian,"Take a large pot or wok, big enough to cook al...",https://www.themealdb.com/images/media/meals/w...,None,https://www.youtube.com/watch?v=IO0issT0Rmc,Chicken,...,1 tsp,To taste,,,,,,None,None,None
1,53110,Sticky Chicken,None,Chicken,Australian,step 1\r\nMake 3 slashes on each of the drumst...,https://www.themealdb.com/images/media/meals/c...,None,https://www.youtube.com/watch?v=o8tz2BOltTg,Chicken drumsticks,...,,,,,,,https://www.bbcgoodfood.com/recipes/sticky-chi...,None,None,2025-11-05 04:28:21
2,52956,Chicken Congee,None,Chicken,Chinese,"STEP 1 - MARINATING THE CHICKEN\r\nIn a bowl, ...",https://www.themealdb.com/images/media/meals/1...,None,https://www.youtube.com/watch?v=kqEfk801E94,Chicken,...,,,,,,,https://sueandgambo.com/pages/chicken-congee,None,None,None
3,52831,Chicken Karaage,None,Chicken,Japanese,"Add the ginger, garlic, soy sauce, sake and su...",https://www.themealdb.com/images/media/meals/t...,None,https://www.youtube.com/watch?v=XivddFddthc,Chicken,...,,,,,,,https://norecipes.com/karaage-recipe,None,None,None
4,52920,Chicken Marengo,None,Chicken,French,Heat the oil in a large flameproof casserole d...,https://www.themealdb.com/images/media/meals/q...,None,https://www.youtube.com/watch?v=U33HYUr-0Fw,Olive Oil,...,,,,,,,https://www.bbcgoodfood.com/recipes/3146682/ch...,None,None,None


In [2]:
# 2. Keep only useful columns (name, area, category, instructions, and ingredient/measure columns)

# These main columns are useful for analysis & prompts
base_cols = ["strMeal", "strCategory", "strArea", "strInstructions"]

# Ingredient and measure columns: strIngredient1..20, strMeasure1..20
ingredient_cols = [f"strIngredient{i}" for i in range(1, 21)]
measure_cols    = [f"strMeasure{i}"    for i in range(1, 21)]

# Combine all columns we want to keep
cols_to_keep = base_cols + ingredient_cols + measure_cols

clean_df = df[cols_to_keep].copy()
clean_df.head(10)


,strMeal,strCategory,strArea,strInstructions,strIngredient1,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,...,strMeasure11,strMeasure12,strMeasure13,strMeasure14,strMeasure15,strMeasure16,strMeasure17,strMeasure18,strMeasure19,strMeasure20
0,Chicken Handi,Chicken,Indian,"Take a large pot or wok, big enough to cook al...",Chicken,Onion,Tomatoes,Garlic,Ginger paste,Vegetable oil,...,2,1 cup,¾ cup,3 tsp Dried,1 tsp,To taste,,,,
1,Sticky Chicken,Chicken,Australian,step 1\r\nMake 3 slashes on each of the drumst...,Chicken drumsticks,Soy Sauce,Honey,Olive Oil,Tomato Puree,Dijon Mustard,...,,,,,,,,,,
2,Chicken Congee,Chicken,Chinese,"STEP 1 - MARINATING THE CHICKEN\r\nIn a bowl, ...",Chicken,Salt,Pepper,Ginger Cordial,Ginger,Spring Onions,...,,,,,,,,,,
3,Chicken Karaage,Chicken,Japanese,"Add the ginger, garlic, soy sauce, sake and su...",Chicken,Ginger,Garlic,Soy sauce,Sake,Granulated sugar,...,,,,,,,,,,
4,Chicken Marengo,Chicken,French,Heat the oil in a large flameproof casserole d...,Olive Oil,Mushrooms,Chicken Legs,Passata,Chicken Stock Cube,Black Olives,...,,,,,,,,,,
5,Spanish Chicken,Chicken,Spanish,step 1\r\nHeat oven to 190C/170C fan/gas 5. Pu...,Chicken Thighs,Onion,Paprika,Lemon,Parsley,Chicken Stock,...,,,,,,,,,,
6,Tandoori chicken,Chicken,Indian,Mix the lemon juice with the paprika and red o...,lemons,paprika,red onions,chicken thighs,vegetable oil,Greek yogurt,...,½ tsp,¼ tsp,,,,,,,,
7,Chicken Couscous,Chicken,Moroccan,Heat the olive oil in a large frying pan and c...,Olive Oil,Onion,Chicken Breast,Ginger,Harissa Spice,Dried Apricots,...,,,,,,,,,,
8,Kung Pao Chicken,Chicken,Chinese,"Combine the sake or rice wine, soy sauce, sesa...",Sake,Soy Sauce,Sesame Seed Oil,Corn Flour,Water,Chicken,...,6 cloves,220g,100g,,,,,,,
9,Chicken Basquaise,Chicken,French,Preheat the oven to 180°C/Gas mark 4. Have the...,Chicken,Butter,Olive Oil,Red Onions,Red Pepper,Chorizo,...,½ tsp,4,Handful,350ml,180g,2,100g,to serve,to serve,


In [3]:
# 3. Build a single clean "ingredients_list" column
# Example: ["1 cup rice", "2 tbsp soy sauce", ...]

def build_ingredients_list(row):
    items = []

    # Loop through ingredient/measure pairs
    for ing_col, meas_col in zip(ingredient_cols, measure_cols):
        ingredient = str(row.get(ing_col, "")).strip()
        measure    = str(row.get(meas_col, "")).strip()

        # Skip blanks or missing ingredients
        if ingredient == "" or ingredient.lower() == "nan":
            continue

        # Combine measure + ingredient (skip measure if it's empty)
        if measure and measure.lower() != "nan":
            items.append(f"{measure} {ingredient}")
        else:
            items.append(ingredient)

    return items

# Create the tidy list column
clean_df["ingredients_list"] = clean_df.apply(build_ingredients_list, axis=1)

# Show only name + ingredients to verify it worked
clean_df[["strMeal", "ingredients_list"]].head(10)



,strMeal,ingredients_list
0,Chicken Handi,"[1.2 kg Chicken, 5 thinly sliced Onion, 2 fine..."
1,Sticky Chicken,"[8 Chicken drumsticks, 2 tblsp Soy Sauce, 1 ta..."
2,Chicken Congee,"[8 oz Chicken, pinch Salt, pinch Pepper, 1 tsp..."
3,Chicken Karaage,"[450 grams Boneless skin Chicken, 1 tablespoon..."
4,Chicken Marengo,"[1 tbs Olive Oil, 300g Mushrooms, 4 Chicken Le..."
5,Spanish Chicken,"[8 Chicken Thighs, 3 sliced thinly Onion, 2 te..."
6,Tandoori chicken,"[2 Juice lemons, 4 tsp paprika, 2 finely chopp..."
7,Chicken Couscous,"[1 tbsp Olive Oil, 1 chopped Onion, 200g Chick..."
8,Kung Pao Chicken,"[2 tbs Sake, 2 tbs Soy Sauce, 2 tbs Sesame See..."
9,Chicken Basquaise,"[1.5kg Chicken, 25g Butter, 6 tblsp Olive Oil,..."


In [4]:
# 4. Drop raw ingredient/measure columns: noisy and unnecessary after tidying

clean_df = clean_df.drop(columns=ingredient_cols + measure_cols)
clean_df.head(10)



,strMeal,strCategory,strArea,strInstructions,ingredients_list
0,Chicken Handi,Chicken,Indian,"Take a large pot or wok, big enough to cook al...","[1.2 kg Chicken, 5 thinly sliced Onion, 2 fine..."
1,Sticky Chicken,Chicken,Australian,step 1\r\nMake 3 slashes on each of the drumst...,"[8 Chicken drumsticks, 2 tblsp Soy Sauce, 1 ta..."
2,Chicken Congee,Chicken,Chinese,"STEP 1 - MARINATING THE CHICKEN\r\nIn a bowl, ...","[8 oz Chicken, pinch Salt, pinch Pepper, 1 tsp..."
3,Chicken Karaage,Chicken,Japanese,"Add the ginger, garlic, soy sauce, sake and su...","[450 grams Boneless skin Chicken, 1 tablespoon..."
4,Chicken Marengo,Chicken,French,Heat the oil in a large flameproof casserole d...,"[1 tbs Olive Oil, 300g Mushrooms, 4 Chicken Le..."
5,Spanish Chicken,Chicken,Spanish,step 1\r\nHeat oven to 190C/170C fan/gas 5. Pu...,"[8 Chicken Thighs, 3 sliced thinly Onion, 2 te..."
6,Tandoori chicken,Chicken,Indian,Mix the lemon juice with the paprika and red o...,"[2 Juice lemons, 4 tsp paprika, 2 finely chopp..."
7,Chicken Couscous,Chicken,Moroccan,Heat the olive oil in a large frying pan and c...,"[1 tbsp Olive Oil, 1 chopped Onion, 200g Chick..."
8,Kung Pao Chicken,Chicken,Chinese,"Combine the sake or rice wine, soy sauce, sesa...","[2 tbs Sake, 2 tbs Soy Sauce, 2 tbs Sesame See..."
9,Chicken Basquaise,Chicken,French,Preheat the oven to 180°C/Gas mark 4. Have the...,"[1.5kg Chicken, 25g Butter, 6 tblsp Olive Oil,..."


In [5]:
# Select only meal names
sample = clean_df["strMeal"].head(10).tolist()
sample



['Chicken Handi',
 'Sticky Chicken',
 'Chicken Congee',
 'Chicken Karaage',
 'Chicken Marengo',
 'Spanish Chicken',
 'Tandoori chicken',
 'Chicken Couscous',
 'Kung Pao Chicken',
 'Chicken Basquaise']

In [6]:
sample2 = clean_df.head(10).to_dict(orient="records")
sample2

[{'strMeal': 'Chicken Handi',
  'strCategory': 'Chicken',
  'strArea': 'Indian',
  'strInstructions': 'Take a large pot or wok, big enough to cook all the chicken, and heat the oil in it. Once the oil is hot, add sliced onion and fry them until deep golden brown. Then take them out on a plate and set aside.\r\nTo the same pot, add the chopped garlic and sauté for a minute. Then add the chopped tomatoes and cook until tomatoes turn soft. This would take about 5 minutes.\r\nThen return the fried onion to the pot and stir. Add ginger paste and sauté well.\r\nNow add the cumin seeds, half of the coriander seeds and chopped green chillies. Give them a quick stir.\r\nNext goes in the spices – turmeric powder and red chilli powder. Sauté the spices well for couple of minutes.\r\nAdd the chicken pieces to the wok, season it with salt to taste and cook the chicken covered on medium-low heat until the chicken is almost cooked through. This would take about 15 minutes. Slowly sautéing the chicken

##  Section 1: Zero-Shot Prompting

In a **zero-shot** prompt, we give the LLM **no examples** — only the name of the meals.  
This shows how the model behaves when it must reason from scratch.

---

### Concept

- *“Zero-shot”* means the model sees **zero examples** of the task beforehand.  
- The model must infer the pattern or logic on its own.  
- This usually leads to inconsistent or hallucinated results, especially when data is complex.

---

###  What to Do

1. Copy the list output you created in the **Preparation** step.  
2. Open ChatGPT (or any LLM environment).  
3. Paste this **prompt template** below.

---

### Prompt Template

```
Here are some meals:

<PASTE DATA HERE>

Classify each meal as healthy or unhealthy.
Explain your reasoning.

```

---

###  Observe and Analyze

While reading the model’s response, pay attention to:

- Does it **invent** ingredients or nutrition facts?  
- Are its **rules consistent** from one meal to another?  
- Does it **confuse** categories like “fried” or “sauce” with unhealthiness automatically?

---

### Example Follow-Up Questions

Ask yourself (or discuss in your notebook):

1. What kinds of reasoning errors did you notice?  
2. Did the model over-generalize or make  assumptions?  
3. How confident was the tone, even when wrong?  
4. What might help reduce these problems?


In [ ]:
#Fetch for the names of meals
# 1. The model added ingredients  assumed nutritional information that was not present in the API results.
# For example, meals that contained no sugar or oil were sometimes described as “high-fat” or “fried,” even when the dataset didn’t include those descriptors.


Provide your findings here:



## Section 2: Few-Shot Prompting with Structured Data (Ingredients + Instructions)

Now that we’ve seen how few-shot prompting works using **meal names only**,  
we’ll expand our examples by including **actual structured information** — ingredients and instructions — from TheMealDB dataset.

This allows the LLM to reason using **factual data** rather than guessing based on names.

---

### Concept

- Adding structured fields like ingredients and instructions helps the model make **evidence-based classifications**.  
- The examples now reflect *both reasoning style* and *data-driven evidence*.  
- This approach reduces hallucinations and supports better transparency.

---

### What to Do

1. Select a few meal entries from your cleaned dataset:


2. Copy the JSON output and use it in the following prompt template.  
   (Each meal record will include `"strMeal"`, `"ingredients_list"`, and `"strInstructions"`.)

---

### Prompt Template

```
You classify meals as Healthy or Unhealthy based on their actual ingredients and instructions.

Examples:
Meal: Grilled Chicken Salad
Ingredients: lettuce, tomato, cucumber, grilled chicken, olive oil
Instructions: Toss vegetables with grilled chicken and drizzle with olive oil.
Label: Healthy
Reason: fresh vegetables and lean protein, minimal oil, not fried.

Meal: Deep Fried Donut
Ingredients: flour, sugar, oil, milk, eggs
Instructions: Mix ingredients, deep fry in oil until golden.
Label: Unhealthy
Reason: high sugar and fat content due to frying.

Now classify the following meals:

<PASTE JSON HERE>

Return:
Meal name
Lable(healthy/unhealthy)
Reason
```

---

### Observe and Analyze

- Does the model use **ingredients and instructions** correctly?  
- Are its **labels** grounded in real evidence?  
- Do explanations cite actual ingredients rather than assumptions?  
- Does performance improve compared to name-only few-shot prompts?

---

### Discussion

1. How did adding structured data change the reasoning style?  
2. Did the model reduce hallucinations or still make unsupported claims?  
3. What trade-offs did you notice between conciseness and accuracy?  
4. How might this affect prompts for real-world data science workflows?

---




In [ ]:
#Fetch the meal's names, ingreduents and instructions
#Organize them into JSON format
# This made the model’s reasoning more grounded and systematic. 
# Instead of relying on stereotypes like “fried food is unhealthy” or guessing nutrition facts,e model began to reference actual ingredients and steps.


Provide your fings here:


## Section 3: Chain-of-Thought (CoT) Prompting

Chain-of-Thought (CoT) prompting encourages the LLM to **reason step-by-step** instead of giving an immediate answer.  
This allows you to see *how* the model reaches its conclusions, improving both transparency and interpretability.

---

### Concept

- CoT prompting asks the model to **explain its thought process** before making a final judgment.  
- This is useful for evaluating **logic, consistency, and data grounding**.  
- In this exercise, you’ll guide the LLM to analyze how **healthy** or **unhealthy** each dish is, based on the ingredients and cooking method.

---

### What to Do

1. You can reuse your existing data sample (e.g., chicken dishes) or fetch a different one using the same API

2. Copy the JSON output and paste it into the prompt template below.

---

### Prompt Template

```
Think step-by-step.

Analyze each meal and determine whether it is prepared in a healthy or unhealthy way.

For each meal:
1. Identify the cooking methods (e.g., fried, baked, grilled, stewed) mentioned in the instructions.
2. List ingredients that add or reduce nutritional value (e.g., vegetables, oils, sugars, processed foods).
3. Explain your reasoning.
4. Conclude with: Healthy or Unhealthy.

Use ONLY the ingredients and instructions in the JSON. Do NOT add any information not present in the data.

<PASTE JSON HERE>
```

---

### Observe and Analyze

- Does the model identify **cooking methods** correctly from the instructions?  
- Are its health-related claims **reasonable and evidence-based**?  
- Does it provide **consistent reasoning** across meals?  
- Does the final decision (Healthy / Unhealthy) align with its step-by-step logic?  

---

### Discussion

1. Did the CoT format help you follow the model’s reasoning?  
2. Did it reduce unexplained or illogical results?  
3. Were the explanations accurate and grounded in data?  
4. What kinds of reasoning errors still appeared?

---

Provide your findings here:
